# Stone Classifier

### Packge Load

In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

from tensorflow.keras import models, layers, losses, optimizers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(777)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


### Prepare Dataset

In [2]:
train_generator = ImageDataGenerator(rescale=1/255.)

dataloader = train_generator.flow_from_directory('./data', target_size=(224,224), 
                                                 classes=['etc', 'chalcopyrite'],
                                                 class_mode='binary', shuffle=True, 
                                                )

Found 7564 images belonging to 2 classes.


### Build Network

In [3]:
base_model = MobileNetV2(weights="imagenet", pooling="avg")
dropout = layers.Dropout(0.5)(base_model.output)
output = layers.Dense(1, activation='sigmoid')(dropout)
model = models.Model(base_model.inputs, output)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### Train Model

In [4]:
model.fit(dataloader, epochs=5)

  ...
    to  
  ['...']
Train for 237 steps
Epoch 1/5
237/237 [==============================] - 41s 172ms/step - loss: 0.5740 - acc: 0.9221
Epoch 2/5
237/237 [==============================] - 29s 124ms/step - loss: 0.4594 - acc: 0.9194
Epoch 3/5
237/237 [==============================] - 29s 124ms/step - loss: 0.3840 - acc: 0.9253
Epoch 4/5
237/237 [==============================] - 30s 125ms/step - loss: 0.3425 - acc: 0.9216
Epoch 5/5
237/237 [==============================] - 30s 125ms/step - loss: 0.3105 - acc: 0.9246
